In [ ]:
# Basic Packages
import numpy as np
import h5py
import logging
import os
import shutil
import gc
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress, pearsonr, spearmanr
import seaborn as sns
from scipy.optimize import curve_fit
import scipy.stats as stats
import pandas as pd
# Physics-related Packages
from astropy.cosmology import Planck15
from astropy.cosmology import Planck15 as cosmo
from scipy.optimize import brentq
import astropy.units as u
from numpy.polynomial.polynomial import polyfit

In [ ]:
import requests
import time

baseUrl = 'http://www.tng-project.org/api/'
headers = {"api-key": "API KEY"}

def get(path, params=None, max_retries=5, retry_delay=2):
    for attempt in range(max_retries):
        try:
            r = requests.get(path, params=params, headers=headers, timeout=30)
            r.raise_for_status()  
            if r.headers.get('content-type') == 'application/json':
                return r.json()
            if 'content-disposition' in r.headers:
                filename = r.headers['content-disposition'].split("filename=")[1]
                with open(filename, 'wb') as f:
                    f.write(r.content)
                return filename
            return r  # fallback
        except (requests.exceptions.RequestException, requests.exceptions.ConnectionError) as e:
            print(f"Attempt {attempt+1}/{max_retries} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
            else:
                raise  

# Issue a request to the API root
r = get(baseUrl)

# Print out all the simulation names
names = [sim['name'] for sim in r['simulations']]
# Get the index of TNG300-1
i = names.index('TNG-Cluster')
# Get the info of simulation Illustris-3
sim = get( r['simulations'][i]['url'] )
sim.keys()

# get the snaps info this simulation
snaps = get(sim['snapshots'])

# Sim Box parameters
Snap_Index = 99 # the snapshots index in the total 100 snapshots taking at different z
BoxSize = sim['boxsize'] # unit: ckpc/h
Redshift = snaps[Snap_Index]['redshift'] # current redshift of our current snap

# Short description of cosmological parameters Planck2015
h_atz = Planck15.H(Redshift).value/100 # unit 100km/[s*Mpc]

LowerMass_lim = 10**3 * h_atz # units 10^ 10 𝑀⊙/ℎ * h
HigherMass_lim = 10**10 * h_atz # units 10^ 10 𝑀⊙/ℎ * h

In [ ]:
fields_1 = ['Coordinates', 'Density', 'ElectronAbundance', 'EnergyDissipation', 'GFM_AGNRadiation']
fields_2 = ['GFM_CoolingRate', 'GFM_Metallicity', 'GFM_Metals', 'InternalEnergy']
fields_3 = ['Masses','StarFormationRate','Velocities']

In [ ]:
cat_df = pd.read_csv('/users/ckong13/data/Chuiyang/TNGCluster_Cutout/fof_halo_to_sub84.csv')
Subhalo_IDs_atTargetSnap = cat_df['Subhalo_ID_At84']
FOF_IDs_PRESORTED = cat_df['FOF_Halo_ID_At84']

In [ ]:
missing_halos = np.array([11314416])

In [ ]:
import re
import time

start_all = time.time() 
snap = 84
field_groups = [fields_1, fields_2, fields_3]

for current_subhalo_id in missing_halos:
# for i,current_subhalo_id in enumerate(Subhalo_IDs_atTargetSnap):
    print(f"🟡 Start downloading {current_subhalo_id}...")

    try:
        #fof_id_presorted = FOF_IDs_PRESORTED[i]
        sub_url_base = f"https://www.illustris-project.org/api/TNG-Cluster/snapshots/{snap}/subhalos/{current_subhalo_id}/"
        sub_info = get(sub_url_base)
        FOF_cutout_url = sub_info['cutouts']['parent_halo']
        match = re.search(r'/halos/(\d+)/', FOF_cutout_url)
        fof_id = int(match.group(1))
        #if fof_id != fof_id_presorted:
        #    print(f'halo {current_subhalo_id} invalid')
        filenames = []

        # dowload data in chunks and then merge data
        for idx, fields in enumerate(field_groups):
            url = FOF_cutout_url + "?gas=" + ",".join(fields)
            fname = f"cutout_{current_subhalo_id}_part{idx+1}.hdf5"
            r = requests.get(url, headers=headers, stream=True)
            r.raise_for_status()
            with open(fname, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
            filenames.append(fname)

        # combine different files
        final_fname = f"cutout_sub{current_subhalo_id}_FOF{fof_id}.hdf5"
        with h5py.File(final_fname, 'w') as fout:
            for fname in filenames:
                with h5py.File(fname, 'r') as fin:
                    for key in fin['PartType0'].keys():
                        fout.create_dataset(f'PartType0/{key}', data=fin['PartType0'][key][:])

        # remover intermediate files
        for fname in filenames:
            os.remove(fname)

        print(f"Finished downloading and merging for {current_subhalo_id}")

    except Exception as e:
        print(f"Failed for {current_subhalo_id} due to {e}")

# running time
elapsed_all = time.time() - start_all
print(f"\n All downloads finished in {elapsed_all:.2f} seconds.")